In [7]:
import ee
import hvplot.xarray
import hvplot.pandas
import xarray

import rasterio as rio
import rioxarray

import xarray
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from tqdm.notebook import tqdm
import math
import json
from datetime import datetime

from concurrent.futures import ThreadPoolExecutor

In [2]:
SHAPEFILE_ROOT = Path('data/ML_training_labels/retrogressive_thaw_slumps/')
OUTFOLDER = Path('data/Sentinel2') 

In [3]:
# Build JoinTable to get date from image_id
labels = map(gpd.read_file, SHAPEFILE_ROOT.glob('*/TrainingLabel*.shp'))
labels = pd.concat(labels).reset_index(drop=True)
labels['image_date'] = pd.to_datetime(labels['image_date'])
id2date = dict(labels[['image_id', 'image_date']].values)

In [4]:
scenes = map(gpd.read_file, SHAPEFILE_ROOT.glob('*/ImageFootprints*.shp'))
scenes = pd.concat(scenes).reset_index(drop=True)
scenes = scenes[scenes.image_id.isin(labels.image_id)]
scenes['image_date'] = scenes.image_id.apply(id2date.get)

# `geedim` with Tileindex Polygons

Best approach so far – TODO: Parallelization

In [5]:
import geedim as gd
gd.Initialize()

BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']

lookback  = pd.Timedelta(days=5)
lookahead = pd.Timedelta(days=5)

s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").select(BANDS)
s2 = gd.MaskedCollection(s2)

In [9]:
def get_scene(i):
  image_id, image_date, geom = scenes.loc[i, ['image_id', 'image_date', 'geometry']]

  minx, miny, maxx, maxy = geom.bounds
  centroid_x = 0.5 * (minx + maxx)
  UTMZone = int(math.ceil((180 + centroid_x) / 6))
  CRS = f'EPSG:326{UTMZone:02d}'
  geom = gpd.GeoSeries(geom, crs='EPSG:4326').to_crs(CRS)[0]
  
  # Define ROI as a ee Geometry
  bounds = ee.Geometry.Polygon(list(geom.exterior.coords),
    proj=CRS, evenOdd=False)

  imgs = s2.search(
    start_date=image_date - lookback,  # Select starting `lookback` days before PSOrthoTile
    end_date=image_date + lookahead,   # Select until `lookahead` days after PSOrthoTile
    region=bounds,
    fill_portion=90,             # Ensure at least 90% of pixels are valid
    custom_filter='CLOUDY_PIXEL_PERCENTAGE < 20'  # Filter Scenes with too many clouds
  )
  
  print(f'Found {len(imgs.properties)} scenes for {image_id}')
  for img in imgs.properties:
    s2_id = img.split('/')[-1]
    outpath = Path(f'data/Sentinel2/{image_id}/{s2_id}.tif')
    if outpath.exists():
      continue
    outpath.parent.mkdir(parents=True, exist_ok=True)
    
    # By default, geedim downloads scenes at native resolution and CRS
    gd.MaskedImage.from_id(img).download(outpath, region=bounds)
    
    # Save S2 image metadata, we might need that later
    with outpath.with_suffix('.json').open('w') as f:
      json.dump(imgs.properties[img], f)

      
with ThreadPoolExecutor(32) as executor:
  list(tqdm(executor.map(get_scene, scenes.index), total=len(scenes)))

Found 2 scenes for 4770635_4572716_2021-08-06_2416


20210806T064629_20210806T064625_T45XWA.tif: |                                               | 0.00/551M (raw) …

Found 3 scenes for 4713120_5272315_2021-07-21_2463
Found 3 scenes for 20190819_080915_1012
Found 1 scenes for 20190820_164144_0f21


  0%|          | 0/154 [00:04<?, ?it/s]

20210721T035541_20210721T035540_T52WEE.tif: |                                               | 0.00/550M (raw) …

...

20200816T050701_20200816T050659_T50XMJ.tif: |                                              | 0.00/77.8M (raw) …

20190911T201021_20190911T201015_T10XFF.tif: |                                              | 0.00/61.4M (raw) …

20210721T071621_20210721T071725_T44WME.tif: |                                               | 0.00/551M (raw) …

20210723T070629_20210723T070721_T43WEV.tif: |                                               | 0.00/440M (raw) …

20200816T050701_20200816T050659_T49XED.tif: |                                              | 0.00/66.9M (raw) …

# The `geedim` way

Best approach so far – TODO: Parallelization

In [ ]:
import geedim as gd
gd.Initialize()

BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']

lookback  = pd.Timedelta(days=5)
lookahead = pd.Timedelta(days=5)

s2 = gd.MaskedCollection.from_name("COPERNICUS/S2_HARMONIZED")

In [ ]:
for (image_id, image_date, region), shapes in tqdm(gdf.groupby(['image_id', 'image_date', 'region'])):
  date = pd.to_datetime(image_date) 

  # Calculate UTM Zone from the region centroid
  centroid_x = 0.5 * (shapes.bounds.minx.min() + shapes.bounds.maxx.max())
  UTMZone = int(math.ceil((180 + centroid_x) / 6))
  CRS = f'EPSG:326{UTMZone:02d}'
  
  # Define ROI as a ee Geometry
  shapes = shapes.to_crs(CRS)
  bounds = shapes.bounds
  bounds = ee.Geometry.Rectangle(
    # xMin, yMin, xMax, yMax
    [bounds.minx.min(), bounds.miny.min(),
     bounds.maxx.max(), bounds.maxy.max()],
    proj=CRS, geodesic=True, evenOdd=False)

  
  imgs = s2.search(
    start_date=date - lookback,  # Select starting `lookback` days before PSOrthoTile
    end_date=date + lookahead,   # Select until `lookahead` days after PSOrthoTile
    region=bounds,
    fill_portion=90,             # Ensure at least 90% of pixels are valid
    custom_filter='CLOUDY_PIXEL_PERCENTAGE < 20'  # Filter Scenes with too many clouds
  )
  
  print(f'Found {len(imgs.properties)} scenes for {image_id}')
  for img in imgs.properties:
    s2_id = img.split('/')[-1]
    outpath = Path(f'data/Sentinel2/{image_id}/{s2_id}.tif')
    outpath.parent.mkdir(parents=True, exist_ok=True)
    
    # By default, geedim downloads scenes at native resolution and CRS
    gd.MaskedImage.from_id(img).download(outpath, region=bounds)
    
    # Save S2 image metadata, we might need that later
    with outpath.with_suffix('.json').open('w') as f:
      json.dump(imgs.properties[img], f)

# The vanilla `ee` way

Works, but is kind of clumsy with regards to getting the data in and out of GDrive.

In [ ]:
ee.Initialize()

DRIVE_FOLDER = 'RTS_S2'
BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']

s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

lookback  = pd.Timedelta(days=14)
lookahead = pd.Timedelta(days=7)

tasklist = []

for (image_id, image_date, region), shapes in tqdm(gdf.groupby(['image_id', 'image_date', 'region'])): 
  centroid_x = 0.5 * (shapes.bounds.minx.min() + shapes.bounds.maxx.max())
  UTMZone = int(math.ceil((180 + centroid_x) / 6))
  CRS = f'EPSG:326{UTMZone:02d}'
  
  shapes = shapes.to_crs(CRS)
  bounds = shapes.bounds
  bounds = ee.Geometry.Rectangle(
    # xMin, yMin, xMax, yMax
    [bounds.minx.min(), bounds.miny.min(),
     bounds.maxx.max(), bounds.maxy.max()],
    proj=CRS, geodesic=True, evenOdd=False)
  
  date = pd.to_datetime(image_date)
  
  imgs = s2
  imgs = imgs.filterDate(date-lookback, date+lookahead)
  imgs = imgs.filterBounds(bounds)
  # imgs = imgs.filter(ee.Filter.intersects(leftField='.all', rightValue=ee.Feature(bounds)))
  imgs = imgs.filter(ee.Filter.contains(leftField='.all', rightValue=ee.Feature(bounds)))
  imgs = imgs.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
  imgs = imgs.select(BANDS)
  imgs = imgs.map(lambda x: x.clip(bounds))
  
  total = imgs.size().getInfo()
  print(f'{image_id}: {total}')
  imgs = imgs.toList(total)

  for i in range(total):
    img = ee.Image(imgs.get(i))
    metadata = img.toDictionary().getInfo()

    timestamp = datetime.fromtimestamp(img.date().getInfo()['value'] / 1000)
    day = timestamp.isoformat().split('T')[0]

    filename = f'{image_id}_{day}'

    task = ee.batch.Export.image.toDrive(
        image=img,
        description=filename,
        scale=10,
        region=bounds,
        fileNamePrefix=filename,
        folder=DRIVE_FOLDER,
        skipEmptyTiles=True,
        crs=CRS,
    )
    task.start()
    tasklist.append(task)
    
    with open(f'data/Sentinel2/{filename}.json', 'w') as f:
      json.dump(metadata, f)
      
  print(f"Done with {image_id}")
  break

# The `wxee` way

We seem to get lots of `nan` bands this way... Not sure why. Let's not use wxee.

In [ ]:
import wxee
wxee.Initialize()

BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']

s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

lookback  = pd.Timedelta(days=14)
lookahead = pd.Timedelta(days=7)

id2paths = {}

for (image_id, image_date, region), shapes in tqdm(gdf.groupby(['image_id', 'image_date', 'region'])):
  if Path(f'data/Sentinel2/{image_id}/').exists():
    # Skip already downloaded tiles
    continue
  
  centroid_x = 0.5 * (shapes.bounds.minx.min() + shapes.bounds.maxx.max())
  UTMZone = int(math.ceil((180 + centroid_x) / 6))
  CRS = f'EPSG:326{UTMZone:02d}'
  
  shapes = shapes.to_crs(CRS)
  bounds = shapes.bounds  
  bounds = ee.Geometry.Rectangle(
    # xMin, yMin, xMax, yMax
    [bounds.minx.min(), bounds.miny.min(),
     bounds.maxx.max(), bounds.maxy.max()],
    proj=CRS, geodesic=True, evenOdd=False)
  
  date = pd.to_datetime(image_date)
  
  imgs = s2
  imgs = imgs.filterDate(date-lookback, date+lookahead)
  imgs = imgs.filterBounds(bounds)
  # imgs = imgs.filter(ee.Filter.intersects(leftField='.all', rightValue=ee.Feature(bounds)))
  imgs = imgs.filter(ee.Filter.contains(leftField='.all', rightValue=ee.Feature(bounds)))
  imgs = imgs.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
  imgs = imgs.map(lambda x: x.clip(bounds))
   
  # acquisition_times = imgs.aggregate_array("system:time_start").getInfo()
  # closest = np.argmin(np.abs(np.asarray(acquisition_times) - date.value / 1e6))
  # img = imgs.toList(imgs, len(acquisition_times)).get(closest)
  
  try:
    data = imgs.wx.to_xarray(scale=10, crs=CRS)
  except ee.EEException:
    print(f'Got EEException for {image_id}, retrying bandwise...')
    data = []
    for band in BANDS:
      data.append(imgs.select([band]).wx.to_xarray(scale=10, crs=CRS))
    data = xarray.merge(data)
    
  if not data.dims:
    # Empty data...
    continue
  
  for time, image in data.groupby('time'):
    day = str(time).split('T')[0]
    
    isnan = np.isnan(image).all()
    if all(isnan[b] for b in isnan):
      continue
    
    outpath = Path(f'data/Sentinel2/{image_id}/{day}.tif')
    outpath.parent.mkdir(parents=True, exist_ok=True)
    image.rio.to_raster(outpath)
  # break
    
  # id2paths[image_id] = paths